In [1]:
from pyflink.table import TableEnvironment, EnvironmentSettings, DataTypes, StreamTableEnvironment

In [2]:
from pyflink.datastream.connectors.kafka import KafkaSource, KafkaOffsetsInitializer

In [3]:
from pyflink.datastream.connectors import KafkaSource

In [4]:
from pyflink.common.serialization import SimpleStringSchema

In [5]:
from pyflink.datastream.formats.json import JsonRowDeserializationSchema

In [6]:
from pyflink.datastream import StreamExecutionEnvironment

In [7]:
from pyflink.common import Types

In [8]:
from pyflink.common.watermark_strategy import WatermarkStrategy

In [9]:
from pyflink.common.configuration import Configuration

In [10]:
from pyflink.common.restart_strategy import RestartStrategies

In [11]:
stream_env = StreamExecutionEnvironment.get_execution_environment()

In [12]:
stream_settings = EnvironmentSettings.new_instance()\
                                    .in_streaming_mode()\
                                    .build()

In [13]:
stream_tbl_env = StreamTableEnvironment.create(stream_execution_environment=stream_env,environment_settings=stream_settings)

In [ ]:
# Put Jars in: D:\Users\kritagya\anaconda3\envs\PyFlink\Lib\site-packages\pyflink

In [14]:
# stream_tbl_env.get_config().set('pipeline.jars','file:///C:/Users/krita/Udemy/PyFlink/flink-sql-connector-kafka-1.17.1.jar')

In [15]:
# stream_env.add_jars('pipeline.jars','file:///C:/Users/krita/Udemy/PyFlink/flink-sql-connector-kafka-1.17.1.jar')

In [16]:
json_types = [Types.STRING(),Types.STRING(),Types.INT(),Types.DOUBLE(),Types.SQL_DATE()]

In [17]:
json_names = ['seller_id','product','quantity','product_price','sales_date']

In [18]:
json_schema = Types.ROW_NAMED(field_names=json_names,field_types=json_types)

In [19]:
json_deserialization_schema = JsonRowDeserializationSchema.Builder()\
                                                        .type_info(type_info=json_schema)\
                                                        .ignore_parse_errors()\
                                                        .build()

In [20]:
kafka_source = KafkaSource.builder() \
    .set_bootstrap_servers('localhost:9092') \
    .set_topics("productsales") \
    .set_group_id("source-demo") \
    .set_starting_offsets(KafkaOffsetsInitializer.earliest()) \
    .set_value_only_deserializer(deserialization_schema=json_deserialization_schema) \
    .build()

In [21]:
kafka_stream_tbl = stream_tbl_env.from_data_stream(data_stream=stream_env.from_source(source=kafka_source,watermark_strategy=WatermarkStrategy.no_watermarks(),source_name='kafka_tbl_source'))

In [ ]:
with kafka_stream_tbl.execute().collect() as results:
    for result in results:
        print(result)

<Row('LNK', 'Toothbrush', 22, 3.99, datetime.date(2021, 7, 1))>
<Row('LNK', 'Dental Floss', 17, 1.99, datetime.date(2021, 7, 1))>
<Row('LNK', 'Toothpaste', 8, 4.99, datetime.date(2021, 7, 1))>
<Row('OMA', 'Toothbrush', 29, 3.99, datetime.date(2021, 7, 1))>
<Row('OMA', 'Toothpaste', 9, 4.99, datetime.date(2021, 7, 1))>
<Row('OMA', 'Dental Floss', 23, 1.99, datetime.date(2021, 7, 1))>
<Row('LNK', 'Toothbrush', 25, 3.99, datetime.date(2021, 7, 2))>
<Row('LNK', 'Dental Floss', 16, 1.99, datetime.date(2021, 7, 2))>
<Row('LNK', 'Toothpaste', 9, 4.99, datetime.date(2021, 7, 2))>
<Row('OMA', 'Toothbrush', 32, 3.99, datetime.date(2021, 7, 2))>
<Row('OMA', 'Toothpaste', 13, 4.99, datetime.date(2021, 7, 2))>
<Row('OMA', 'Dental Floss', 18, 1.99, datetime.date(2021, 7, 2))>
<Row('LNK', 'Toothbrush', 20, 3.99, datetime.date(2021, 7, 3))>
<Row('LNK', 'Dental Floss', 15, 1.99, datetime.date(2021, 7, 3))>
<Row('LNK', 'Toothpaste', 11, 4.99, datetime.date(2021, 7, 3))>
<Row('OMA', 'Toothbrush', 31, 3.9

In [ ]:
# stream_tbl_env.create_table("MyTable", TableDescriptor.for_connector("datagen")
#     .schema(Schema.new_builder()
#                   .column("f0", DataTypes.STRING())
#                   .build())
#     .option("rows-per-second", 10)
#     .option("fields.f0.kind", "random")
#     .build())

In [ ]:
# ds = stream_env.from_source(source=kafka_source,watermark_strategy=WatermarkStrategy.no_watermarks(),source_name='kafka_tbl_source')

In [ ]:
# with ds.execute_and_collect() as results:
#     for result in results:
#         print(result)